In [ ]:
!pip install rasterio geopandas pystac-client planetary-computer plotly matplotlib scipy

In [ ]:
from google.colab import files
import rasterio

uploaded = files.upload()
tif_file = list(uploaded.keys())[0]

with rasterio.open(tif_file) as src:
    bounds = src.bounds
    crs = src.crs
    width = src.width
    height = src.height
    resolution = src.res[0]  # should be 10.0 for Sentinel

print(f"Bounds: {bounds}")
print(f"Resolution: {resolution} m | Size: {width} x {height}")


In [ ]:
from pystac_client import Client
import planetary_computer

stac = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

from rasterio.warp import transform_bounds

# Convert bounding box from UTM (meters) to WGS84 (lat/lon)
bounds_wgs84 = transform_bounds(src.crs, "EPSG:4326", *bounds)

# Query STAC with WGS84 bounds
search = stac.search(
    collections=["cop-dem-glo-30"],
    bbox=bounds_wgs84,  # now in degrees!
    max_items=1
)

# Get the DEM item and signed URL
item = next(search.items())  # use `.items()` instead of `.get_items()` (new API)
asset = item.assets["data"]
signed_href = planetary_computer.sign(asset.href)
print("✅ Found DEM asset:\n", signed_href)


In [ ]:
import numpy as np
from pyproj import CRS
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
from rasterio.windows import from_bounds
from rasterio.warp import transform_bounds
from scipy.ndimage import gaussian_filter, sobel
import rasterio

# Open TIF and get metadata
with rasterio.open(tif_file) as src:
    tif_crs = src.crs
    tif_bounds = src.bounds
    tif_width = src.width
    tif_height = src.height
    tif_bounds_wgs84 = transform_bounds(tif_crs, "EPSG:4326", *tif_bounds)

# Open DEM and check if it's projected
with rasterio.open(signed_href) as dem_src:
    dem_crs = dem_src.crs
    print("DEM CRS:", dem_crs)
    print("Is projected:", dem_crs.is_projected)

    if dem_crs.is_projected:
        # DEM is already in meters — no reprojection needed
        print("✅ DEM is already in meters. Skipping reprojection.")

        window = from_bounds(*tif_bounds, transform=dem_src.transform)
        transform = dem_src.window_transform(window)
        dem_data = dem_src.read(1, window=window, out_shape=(tif_height, tif_width))

    else:
        # DEM is in degrees — reproject to UTM
        print("🔁 DEM is in degrees. Reprojecting to UTM.")

        # Compute UTM CRS from TIF center
        lon_center = (tif_bounds_wgs84[0] + tif_bounds_wgs84[2]) / 2
        lat_center = (tif_bounds_wgs84[1] + tif_bounds_wgs84[3]) / 2
        utm_zone = int((lon_center + 180) / 6) + 1
        is_northern = lat_center >= 0
        utm_crs = CRS.from_dict({
            'proj': 'utm',
            'zone': utm_zone,
            'datum': 'WGS84',
            'south': not is_northern
        })
        print("Reprojecting DEM to:", utm_crs)

        # Convert TIF bounds to UTM
        tif_bounds_utm = transform_bounds(tif_crs, utm_crs, *tif_bounds)

        # Warp the DEM
        with WarpedVRT(dem_src, crs=utm_crs, resampling=Resampling.bilinear) as vrt:
            window = from_bounds(*tif_bounds_utm, transform=vrt.transform)

            if window.width == 0 or window.height == 0:
                raise ValueError("❌ DEM window is empty after reprojection. Check bounds or STAC match.")

            transform = vrt.window_transform(window)
            dem_data = vrt.read(1, window=window, out_shape=(tif_height, tif_width))

# ✅ Slope calculation (always in meters now)
pixel_size = abs(transform.a)
smoothed = gaussian_filter(dem_data, sigma=1)
dx = sobel(smoothed, axis=1) / (8.0 * pixel_size)
dy = sobel(smoothed, axis=0) / (8.0 * pixel_size)
slope_rad = np.arctan(np.sqrt(dx**2 + dy**2))
slope = np.degrees(slope_rad)

if slope.size == 0:
    raise ValueError("❌ Computed slope array is empty. Likely DEM extraction failed.")

print("✅ Slope sanity check:")
print(f"Min: {np.nanmin(slope):.2f}°")
print(f"Max: {np.nanmax(slope):.2f}°")
print(f"Mean: {np.nanmean(slope):.2f}°")
print(f"Pixels > 45°: {np.sum(slope > 45)} / {slope.size}")


In [ ]:
print("TIF CRS:", crs)
print("Is projected:", crs.is_projected)


In [ ]:
import rasterio

with rasterio.open(tif_file) as src:
    print("Band count:", src.count)
    print("CRS:", src.crs)
    print("Shape:", src.height, "x", src.width)
    print("Descriptions:", src.descriptions)


In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

def normalize(image, lower=2, upper=98):
    img = image.astype(np.float32)
    min_val = np.percentile(img, lower)
    max_val = np.percentile(img, upper)
    img = np.clip((img - min_val) / (max_val - min_val), 0, 1)
    return (img * 255).astype(np.uint8)

with rasterio.open(tif_file) as src:
    rgb_raw = src.read([1, 2, 3])  # B4, B3, B2
    height, width = rgb_raw.shape[1], rgb_raw.shape[2]

# Normalize each channel and combine
rgb_img = np.stack([normalize(b) for b in rgb_raw], axis=-1)  # Shape: (H, W, 3)

# Plot with pixel coordinate grid
plt.figure(figsize=(8, 8))
plt.imshow(rgb_img)
plt.title("RGB Composite (B4-B3-B2)")

# Add tick marks every 50 pixels
tick_spacing = 50
plt.xticks(np.arange(0, width + 1, tick_spacing))
plt.yticks(np.arange(0, height + 1, tick_spacing))

# Grid lines every 50 pixels
plt.grid(True, which='both', linestyle=':', color='lightgray')

# Axis labels
plt.xlabel("Pixel X")
plt.ylabel("Pixel Y")

# Optional: Flip Y to match image layout (top-left origin)
# plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 5))

# Get height and width of the DEM
height, width = dem_data.shape

# --- Elevation Plot ---
plt.subplot(1, 2, 1)
plt.title("Elevation (DEM)")
plt.imshow(dem_data, cmap='terrain')
plt.xlabel("Pixel X")
plt.ylabel("Pixel Y")
plt.xticks(np.linspace(0, width - 1, 6), labels=[f"{int(x)}" for x in np.linspace(0, 500, 6)])
plt.yticks(np.linspace(0, height - 1, 6), labels=[f"{int(y)}" for y in np.linspace(0, 500, 6)])
plt.colorbar(label='m')

# --- Slope Plot ---
plt.subplot(1, 2, 2)
plt.title("Slope (degrees)")
plt.imshow(slope, cmap='viridis')
plt.xlabel("Pixel X")
plt.ylabel("Pixel Y")
plt.xticks(np.linspace(0, width - 1, 6), labels=[f"{int(x)}" for x in np.linspace(0, 500, 6)])
plt.yticks(np.linspace(0, height - 1, 6), labels=[f"{int(y)}" for y in np.linspace(0, 500, 6)])
plt.colorbar(label='°')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# x and y can just be regular index ranges
x = np.arange(dem_data.shape[1])
y = np.arange(dem_data.shape[0])

# No need to flip the data itself!
fig = go.Figure(data=[go.Surface(z=dem_data, x=x, y=y, colorscale='viridis')])

# Flip just the view — autorange='reversed' flips axis without touching data
fig.update_layout(
    title='3D Terrain Elevation',
    scene=dict(
        zaxis_title='Elevation (m)',
        xaxis_title='X',
        yaxis_title='Y',
        xaxis=dict(autorange='reversed')  # This is the key
    )
)

fig.show()


In [ ]:
print("Runway Suitability Check:")
print(f"Mean Slope: {np.nanmean(slope):.2f}°")
print(f"Max Slope: {np.nanmax(slope):.2f}°")
print(f"Flat Area (% < 2° slope): {(np.sum(slope < 2) / slope.size) * 100:.2f}%")
